<a href="https://colab.research.google.com/github/AnandVishnuG/ImageCaptioningLSTM/blob/main/ImageCaptioningLSTMTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.6 MB/s eta 0:00:00


In [2]:
from matplotlib import pyplot as plt
from google.colab import files, drive
from tqdm import tqdm 
from PIL import Image
from pickle import dump, load
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Reshape
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences, to_categorical
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from keras.preprocessing.image import ImageDataGenerator

import itertools
import numpy as np
import pandas as pd
import os
import bert_score

In [3]:
# Mount Google drive for accessing files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a dictionary of images as keys and each image's descriptions as items
def data_to_dictionary(data):
    descriptions = {}
    for idx, row in data.iterrows():
        if row["image"] not in descriptions:
            descriptions.setdefault(row["image"], []).append(row["caption"])
        else:
            descriptions[row["image"]].append(row["caption"])
    return descriptions
#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc    
# Converting captions to tokens
def list_to_tokens(desc_list):
  tokens = []
  for desc in desc_list:
    tokens.append(word_tokenize(desc))
  return tokens


In [ ]:
# Read Captions file 
captions = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/captions_clean.txt",encoding="cp1252")
# Create descriptions dictionary containing image keys and corresponding captions
descriptions = data_to_dictionary(captions)


In [ ]:
# Extract image features using Xception model and save it
def extract_features():
        model = Xception( include_top=False, pooling='avg' )
        # datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
        # datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)
        datagen = ImageDataGenerator(brightness_range=(0.5, 1.5))
            
        features = {}
        for img in tqdm(os.listdir("/content/drive/MyDrive/Colab Notebooks/data/Images")):
            filename = "/content/drive/MyDrive/Colab Notebooks/data/Images/"  + img
            image = Image.open(filename)
            image = np.expand_dims(image, axis=0)
             # Generate augmented images
            aug_iter = datagen.flow(image, batch_size=1)
            aug_images = [next(aug_iter)[0].astype(np.uint8) for _ in range(2)]
            aug_images = [aug_image/127.5 - 1 for aug_image in aug_images]

            for idx, aug_image in enumerate(aug_images):
              aug_image = np.expand_dims(aug_image, axis=0)
              feature = model.predict(aug_image, verbose=0)
              features[f"{img}_{idx}"] = feature
        return features
features = extract_features()
dump(features, open("features_aug2_brightness.p", "wb"))


In [ ]:
# Generate augmented images
datagen = ImageDataGenerator(brightness_range=(0.5, 1.5),channel_shift_range=50)
        
image = Image.open("/content/drive/MyDrive/Colab Notebooks/data/Images/111537222_07e56d5a30.jpg")
image = np.expand_dims(image, axis=0)
            
aug_iter = datagen.flow(image, batch_size=1)
aug_images = [next(aug_iter)[0].astype(np.uint8) for _ in range(2)]
aug_images = [aug_image/127.5 - 1 for aug_image in aug_images]
# Display images
for img in aug_images:
  plt.imshow(img)
  plt.show(block=True)

In [ ]:
def load_photos(filename):
    file = open(filename, 'r')
    photos = file.read().split("\n")[:-1]
    return photos
def load_descriptions(photos):
    d = {}
    for photo in photos:
        if photo in descriptions.keys():
            if photo not in d.keys():
                l = []            
                for desc in descriptions[photo]:
                    l.append("<start> " + desc + " <end>" ) 
                d.setdefault(photo+"_0", l)
                d.setdefault(photo+"_1", l)
    return d
def load_features(photos, features):
    #loading all features
    all_features = features 
    f={}
    #selecting only needed features
    # features = {k:all_features.get(k) for k in photos}
    for k in photos:
      f[k+"_0"] = all_features[k+"_0"]
      f[k+"_1"] = all_features[k+"_1"]
    return f
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer
#calculate maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)
    
# Training data from file
train_photos   = load_photos("/content/drive/MyDrive/Colab Notebooks/data/Flickr_8k.trainImages.txt")
print(len(train_photos))
# Training description from Descriptions dictionary
train_desc     = load_descriptions(train_photos)
print(len(train_desc))
# Extract training image features from extracted features
train_features = load_features(train_photos, features)
print(len(train_features))

# Validation data from file
val_photos   = load_photos("/content/drive/MyDrive/Colab Notebooks/data/Flickr_8k.devImages.txt")
# Training description from Descriptions dictionary
val_desc     = load_descriptions(val_photos)
# Extract training image features from extracted features
val_features = load_features(val_photos, features)

# Convert training descriptions to list 
desc_list = dict_to_list(train_desc)
# Create a tokenizer which has all training unique words with coresponding number
tokenizer = create_tokenizer(train_desc)
vocab_size = len(tokenizer.word_counts) + 1
max_length = max_length(descriptions)
max_length

In [ ]:
def data_generator_batch_augmented(descriptions, features, tokenizer, max_length, batch_size=32):
    counter = 0
    features_list = list()
    desc_list = list()
    while True:
        for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key][0]
            features_list.append(feature)
            if desc_list == []:
              desc_list = description_list
            else:
              desc_list.append(description_list)
            counter += 1
            if counter == batch_size:
                counter = 0
                input_image, input_sequence, output_word = create_sequences_batch_augmented(tokenizer, max_length, desc_list, features_list)
                yield [[input_image, input_sequence], output_word]
                features_list = list()
                desc_list = list()            

def create_sequences_batch_augmented(tokenizer, max_length, desc_list, feature, batch_size = 32):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for idx, desc in enumerate(desc_list):
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # seq = text_to_sequence(tokenizer, desc)
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(feature[idx//5])
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

#You can check the shape of the input and output for your model
# [a,b],c = next(data_generator(train_desc, features, tokenizer, max_length))
# a.shape, b.shape, c.shape
#((47, 2048), (47, 32), (47, 7577))
#((50, 2048), (50, 38), (50, 7321))

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import add, SimpleRNN
from keras.layers import LSTM

# define the captioning model
def define_model(vocab_size, max_length):
    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    # inputs1 = Input((299,299,1))
    # Including the Xception model without the final layer
    # fe1 = Xception(include_top = False, pooling='avg' )(inputs1)    
    # Setting only the final layers as trainable
    # for layer in fe1.layers[:-36]:
    #   layer.trainable = False
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu',kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(fe1)
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256, kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(se2)
    se4 = Dropout(0.5)(se3)
    # Merging both models
    decoder1 = add([fe2, se4])
    decoder2 = Dense(128, activation='relu', kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001))(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.reset_states()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
# train model
print('Dataset: ', len(train_photos))
print('Descriptions: train=', len(train_desc))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)
# epochs = 10
# steps = len(train_desc)
# steps_val = len(val_desc)

# for i in range(epochs):
#     generator_train = data_generator(train_desc, train_features, tokenizer, max_length)
#     generator_val = data_generator(val_desc, val_features, tokenizer, max_length)
#     model.fit(generator_train, validation_data= generator_val, epochs=1, steps_per_epoch= steps, validation_steps= steps_val, verbose=1, callbacks = [tensorboard_callback])
#     model.save("model_" + str(i) + ".h5")
epochs = 10
BATCH = 1
steps = len(train_desc)// BATCH
steps_val = len(val_desc) // BATCH

for i in range(epochs):
    generator_train = data_generator_batch_augmented(train_desc, train_features, tokenizer, max_length, BATCH)
    generator_val = data_generator_batch_augmented(val_desc, val_features, tokenizer, max_length, BATCH)
    model.fit(generator_train, validation_data= generator_val, epochs=1, steps_per_epoch= steps, validation_steps= steps_val, verbose=1)
    model.save("model_" + str(i) + ".h5")

In [ ]:
from google.colab import files
files.download('/content/features_aug2.p')
files.download('/content/model_9.h5')

In [ ]:
from tensorflow.keras.applications import Xception

def extract_features(filename, model):
        try:
            image = Image.open(filename)

        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    
    # for word, index in tokenizer.items():
    #     if index == integer:
    #         return word
    return None


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # sequence = text_to_sequence(tokenizer, in_text)
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text.replace("start ","").replace(" end","").capitalize() + "."


#path = 'Flicker8k_Dataset/111537222_07e56d5a30.jpg'
max_length = 38
# tokenizer = load(open("/content/tokenizer_keras_reg.p","rb"))#/content/drive/MyDrive/Colab Notebooks/data/Models/tokenizer.p
model = load_model('/content/model_9 (1).h5')
# model = load_model('/content/model_9.h5')#'/content/drive/MyDrive/Colab Notebooks/data/model_19_kerasTokenizer.h5')#/content/drive/MyDrive/Colab Notebooks/data/Models/model_9.h5')#/content/drive/MyDrive/Colab Notebooks/data/model_9.h5
xception_model = Xception(include_top=False, pooling="avg")

photo = extract_features("/content/drive/MyDrive/Colab Notebooks/data/Images/1000268201_693b08cb0e.jpg", xception_model)#/content/drive/MyDrive/Colab Notebooks/data/Images/111537222_07e56d5a30.jpg
img = Image.open("/content/drive/MyDrive/Colab Notebooks/data/Images/1000268201_693b08cb0e.jpg")

description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(img)

In [ ]:
from pandas import read_csv
import random
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score


smooth_func = SmoothingFunction().method1

file = open('/content/drive/MyDrive/Colab Notebooks/data/Flickr_8k.testImages.txt',"r")
test_images = []

for filename in file.read().split("\n")[:-1]:
    test_images.append(filename)
# model = load_model("models\\model2_19.h5")
for i in range(0,5):
    rand_idx = random.randint(0, len(test_images)-1)
    fname = "/content/drive/MyDrive/Colab Notebooks/data/Images/" + test_images[rand_idx]
    photo = extract_features (fname, xception_model)
    img = Image.open(fname)
    disc = generate_desc(model, tokenizer, photo, max_length)
    print("\n\n")
    # Prediction
    print("Prediction: {}".format(disc))
    print()
    # Description
    score = sentence_bleu(descriptions[test_images[rand_idx]], disc, smoothing_function=smooth_func)
    for desc in descriptions[test_images[rand_idx]]:
        print("Reference: {} ".format(desc))
    print("BLEU: {}".format(score))  
    # tokenizer.tokenize()
    # print(descriptions[test_images[rand_idx]])
    # print("METEOR: {}".format(meteor_score(descriptions[test_images[rand_idx]], str(description))))
    # Sentence BLEU
    # total_score.append(score)
    plt.figure()
    plt.imshow(img)
    plt.show(block=True)

In [ ]:
file = open("/content/drive/MyDrive/Colab Notebooks/data/Flickr_8k.testImages.txt", "r")
predictions = []
# Extracting predictions
for filename in file.read().split("\n")[:-1]:
    photo = extract_features("/content/drive/MyDrive/Colab Notebooks/data/Images/" + filename, xception_model)
    description = generate_desc(model, tokenizer, photo, max_length)
    predictions.append(description)
predictions    

In [ ]:
from bert_score import score as bscore
file = open("/content/drive/MyDrive/Colab Notebooks/data/Flickr_8k.testImages.txt", "r")
bert_score_results = {}
for idx, filename in enumerate(file.read().split("\n")[:-1]):
  scores = []
  for desc in descriptions[filename]:
    _, _, score = bscore([desc], [predictions[idx]], lang='en', verbose=False)
    scores.append(score.item())
  mean_score = np.mean(scores)
  bert_score_results.setdefault(filename, (predictions[idx],mean_score))
bert_score_results